In [1]:
from pyspark import *

### Data Summary
https://archive.ics.uci.edu/ml/datasets/Adult

Attribute Information:

Listing of attributes: 

>50K, <=50K. 

#### Feature 1 =  age: continuous. 
#### Feature 2 = workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
#### Feature 3 = fnlwgt: continuous. 
#### Feature 4 = education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
#### Feature 5 = education-num: continuous. 
#### Feature 6 = marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
#### Feature 7 = occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
#### Feature 8 = relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
#### Feature 9 = race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
#### Feature 10 = sex: Female, Male. 
#### Feature 11 = capital-gain: continuous. 
#### Feature 12 = capital-loss: continuous. 
#### Feature 13 = hours-per-week: continuous. 
#### Feature 14 = native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [3]:
#Step 1: Read the table from the spark data
census = spark.read.table("adult_csv")
#Step2 : Reading the Table Schema
census.printSchema()
#Step 3: Rading the column names
census_col = census.columns
census.dtypes
#census = census.withColumnRenamed("age", "Age")

root
-- age: integer (nullable = true)
-- employer_type: string (nullable = true)
-- numf1: integer (nullable = true)
-- education: string (nullable = true)
-- numf2: integer (nullable = true)
-- marital: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- gender: string (nullable = true)
-- capitalgain: integer (nullable = true)
-- capitalloss: integer (nullable = true)
-- hr_per_wk: integer (nullable = true)
-- region: string (nullable = true)
-- income: string (nullable = true)

In [4]:
census.count()

Out[6]: 48842

In [5]:
# printing limited columns we use select keyword
census.select("employer_type", "marital", "income").show() # Our 2nd transformation (4 >> 3 columns)

+----------------+------------------+------+
 employer_type| marital|income|
+----------------+------------------+------+
 Private| Never-married| <=50K|
 Private|Married-civ-spouse| <=50K|
 Local-gov|Married-civ-spouse| >50K|
 Private|Married-civ-spouse| >50K|
 ?| Never-married| <=50K|
 Private| Never-married| <=50K|
 ?| Never-married| <=50K|
Self-emp-not-inc|Married-civ-spouse| >50K|
 Private| Never-married| <=50K|
 Private|Married-civ-spouse| <=50K|
 Private|Married-civ-spouse| >50K|
 Federal-gov|Married-civ-spouse| <=50K|
 Private| Never-married| <=50K|
 ?|Married-civ-spouse| <=50K|
 Private|Married-civ-spouse| >50K|
 Private|Married-civ-spouse| >50K|
 State-gov| Never-married| <=50K|
 Private|Married-civ-spouse| <=50K|
 Private| Widowed| <=50K|
 Private|Married-civ-spouse| >50K|
+----------------+------------------+------+
only showing top 20 rows

In [6]:
# check for all missing values in Dataframe
import pyspark.sql.functions as F
#cols = [count(c=='?')  for c in census.columns]
cout = [census[census[c]=="?"].count() for c in census.columns]
print(census.columns)
cout

['age', 'employer_type', 'numf1', 'education', 'numf2', 'marital', 'occupation', 'relationship', 'race', 'gender', 'capitalgain', 'capitalloss', 'hr_per_wk', 'region', 'income']
Out[8]: [0, 2799, 0, 0, 0, 0, 2809, 0, 0, 0, 0, 0, 0, 857, 0]

In [7]:
step1 = [F.when(~F.col(x).isin("?","NULL", "NA", "NaN"), F.col(x)).alias(x)  for x in census.columns] 
step2 = census.select(*step1).dropna(how='any')
step3 = step2.drop('numf1','numf2') 

#KPI 1

In [9]:
#KPI1
display(step3.select(['age','income']))

age,income
25,<=50K
38,<=50K
28,>50K
44,>50K
34,<=50K
63,>50K
24,<=50K
55,<=50K
65,>50K
36,<=50K


In [10]:
# Step 4.1 Selecting Columns
step3.select('education','income').head(5)

Out[13]: [Row(education='11th', income='<=50K'),
 Row(education='HS-grad', income='<=50K'),
 Row(education='Assoc-acdm', income='>50K'),
 Row(education='Some-college', income='>50K'),
 Row(education='Some-college', income='<=50K')]

In [11]:
#df = spark.read.options(header =True, inferSchema = True).csv("mtcars")

In [12]:
#Filtering data two ways tp do that second one is slower
step3[(step3.age>30) & (step3.age<35)].show(5)
#step3.filter((step3.age>30) & (step3.age<35)).show(5)

+---+----------------+------------+------------------+---------------+-------------+-----+------+-----------+-----------+---------+-------------+------+
age| employer_type| education| marital| occupation| relationship| race|gender|capitalgain|capitalloss|hr_per_wk| region|income|
+---+----------------+------------+------------------+---------------+-------------+-----+------+-----------+-----------+---------+-------------+------+
 34| Private| 10th| Never-married| Other-service|Not-in-family|White| Male| 0| 0| 30|United-States| <=50K|
 34| Private| Bachelors|Married-civ-spouse| Tech-support| Husband|White| Male| 0| 0| 47|United-States| >50K|
 34| Private|Some-college| Never-married| Other-service| Own-child|Black|Female| 0| 0| 35|United-States| <=50K|
 32|Self-emp-not-inc|Some-college| Never-married| Prof-specialty|Not-in-family|White| Male| 0| 0| 60|United-States| <=50K|
 34| Local-gov| Bachelors|Married-civ-spouse|Exec-managerial| Husband|White| Male| 3103| 0| 50|United-States| >50K|
+---+----------------+------------+------------------+---------------+-------------+-----+------+-----------+-----------+---------+-------------+------+
only showing top 5 rows

In [13]:
#Addning the new column
#step3.withColumn('new',step3.capitalgain-step3.capitalloss).show(5)

In [14]:
# categorizing age group
def categorizer(age):
  if age < 30:
    return "young"
  elif age < 45:
    return "middle"
  elif age < 60:
    return "senior"
  elif age < 60:
    return "top senior"
  else: 
    return "retired"
# i have to write udf(user defined functions and below is one way to do so)
# udf(user defined function) the function and its type and then use withcolumn to apply the transformation  
from pyspark.sql.functions import udf
from pyspark.sql.types import *
bucket_udf = udf(categorizer, StringType() )     
step3 = step3.withColumn("age group", bucket_udf("age"))


In [15]:
# categorizing hrs/week
def worktimecategorizer(hours):
  if hours <= 25:
    return "Temp"
  elif hours <= 45:
    return "Full time"
  else:
    return "Overtime"
# i have to write udf(user defined functions and below is one way to do so)
from pyspark.sql.functions import udf
from pyspark.sql.types import *
workbucket_udf = udf(worktimecategorizer, StringType())     
step3 = step3.withColumn("workhour", workbucket_udf("hr_per_wk")).drop('hr_per_wk').withColumnRenamed('workhour','hr/wk')

In [16]:
Asia=['China','Hong','India','Iran','Cambodia','Japan', 'Laos',
        ' Philippines' ,' Vietnam' ,' Taiwan', ' Thailand']
North_America = ['Canada','United-States','Puerto-Rico' ]

Europe = ['England' ,'France', 'Germany' ,'Greece','Holand-Netherlands','Hungary',
            'Ireland','Italy','Poland','Portugal','Scotland','Yugoslavia']

Latin_and_SouthAmerica = ['Columbia','Cuba','Dominican-Republic','Ecuador',
                             'El-Salvador','Guatemala',' Haiti',' Honduras',
                             'Mexico','Nicaragua','Outlying-US(Guam-USVI-etc)','Peru',
                             'Jamaica','Trinadad&Tobago']
Other= ['South']

def regionlist(countries):
  if countries in Asia:
    return "Asia"
  if countries in North_America:
    return "North America"
  if countries in Europe:
    return "Europe"
  if countries in Latin_and_SouthAmerica:
    return "Latin & South America"
  else:
    return "Others"
# udf(user defined function) the function and its type and then use withcolumn to apply the transformation  
region_udf = udf(regionlist,StringType())
step3 = step3.withColumn("Reg",region_udf("region")).drop('region').withColumnRenamed('Reg','region')


In [17]:
def employementtype(job):
  if(job=='Never-worked'or job=='Without-pay'):
    return "Unemployed"
  if(job=='State-gov' or job=='Local-gov'):
    return "Govt"
  if(job=='Self-emp-inc' or job=='Self-emp-not-inc'):
    return "self_emp"
  else:
    return(job)
# udf(user defined function) the function and its type and then use withcolumn to apply the transformation  
etype_udf = udf(employementtype,StringType())
step3 = step3.withColumn("employement", etype_udf("employer_type")).drop('employer_type').withColumnRenamed('employement','employer type')
#employementtype('Never-worked')



In [18]:
def maritalstat(mar):
  #mar= as.character(mar)
  
  if mar in ['Separated','Widowed','Divorced']:
    return "Not-Married"
  elif mar== 'Never-married':
    return "Never-married"
  else: 
    return "Married"
# udf(user defined function) the function and its type and then use withcolumn to apply the transformation
marital_udf = udf(maritalstat,StringType())
step3 = step3.withColumn("status", marital_udf("marital")).drop('marital').withColumnRenamed('status','marital')
#maritalstat('Separated')

In [19]:
step3.printSchema()
step3.cache()


root
-- age: integer (nullable = true)
-- education: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- gender: string (nullable = true)
-- capitalgain: integer (nullable = true)
-- capitalloss: integer (nullable = true)
-- income: string (nullable = true)
-- age group: string (nullable = true)
-- hr/wk: string (nullable = true)
-- region: string (nullable = true)
-- employer type: string (nullable = true)
-- marital: string (nullable = true)

Out[31]: DataFrame[age: int, education: string, occupation: string, relationship: string, race: string, gender: string, capitalgain: int, capitalloss: int, income: string, age group: string, hr/wk: string, region: string, employer type: string, marital: string]

In [20]:
census_clean = step3

# Basic cleaning is done
#Time for Some Visualizations and analyse my data

#KPI 2

In [23]:
#How to use groupby and aggregate functions alongwith it
#KPI2
daf02 = census_clean.groupby(['education']).agg({'capitalgain':'mean','capitalloss':'mean'})
display(daf02.sort("avg(capitalgain)", ascending  = False))

education,avg(capitalloss),avg(capitalgain)
Prof-school,239.3936305732484,10981.208917197453
Doctorate,236.18566176470588,6068.02205882353
Masters,167.23070803500397,2567.1495624502786
Bachelors,124.98375165125495,1744.5273447820343
Preschool,47.09722222222222,843.8333333333334
Assoc-voc,69.66462480857581,805.1408882082695
HS-grad,69.4066833524995,581.5359534600555
Assoc-acdm,92.2733908427339,572.4452554744526
Some-college,73.83149813112436,570.4425699565612
5th-6th,68.86191536748329,387.0423162583519


#KPI 3

In [25]:
#KPI3
daf03 = census_clean.groupBy(['income','age group']).agg({'capitalgain':'mean', 'capitalloss':'mean'})
display(daf03.sort("avg(capitalgain)", ascending = False))

income,age group,avg(capitalloss),avg(capitalgain)
>50K,retired,217.38341346153845,6202.330528846154
>50K,senior,192.30252100840337,4451.461951447245
>50K,middle,191.16942841184138,3417.7482777881214
>50K,young,190.2149791955617,2985.9542302357836
<=50K,retired,70.08077879038939,315.3384424192212
<=50K,senior,64.9410719573586,170.50488599348535
<=50K,middle,58.87821434275675,155.7339881808018
<=50K,young,39.98693605972087,97.85491723466407


#KPI 4

In [27]:
#KPI 4
daf04 = census_clean.groupBy('race','income').agg({'income':'count'})
display(daf04.sort("count(income)", ascending = False))

race,income,count(income)
White,<=50K,28696
White,>50K,10207
Black,<=50K,3694
Asian-Pac-Islander,<=50K,934
Black,>50K,534
Amer-Indian-Eskimo,<=50K,382
Asian-Pac-Islander,>50K,369
Other,<=50K,308
Amer-Indian-Eskimo,>50K,53
Other,>50K,45


#KPI stat

In [29]:
census_clean.groupBy('income').agg({'age':'mean'}).show()

+------+------------------+
income| avg(age)|
+------+------------------+
 <=50K|36.749426706650205|
 >50K| 44.00606709493219|
+------+------------------+

#KPI 5

In [31]:
#KPI 5
daf05 = census_clean.groupBy(['gender','relationship']).agg({'age':'mean'})
#daf05.describe
display(daf05.sort("avg(age)", ascending = False))

gender,relationship,avg(age)
Male,Husband,43.35788909724082
Female,Unmarried,40.79030933479332
Female,Wife,39.97464114832536
Female,Not-in-family,39.738913525498894
Male,Unmarried,38.70044052863436
Female,Other-relative,36.90327868852459
Male,Not-in-family,36.58139904610493
Female,Husband,34.0
Male,Other-relative,30.687415426251693
Male,Wife,29.0


#KPI 6

In [33]:
#KPI 6
daf06 = census_clean.groupBy('education','income').agg({'income':'count'})
display(daf06.sort('count(income)',ascending = False))

education,income,count(income)
HS-grad,<=50K,12367
Some-college,<=50K,7909
Bachelors,<=50K,4392
Bachelors,>50K,3178
HS-grad,>50K,2416
Some-college,>50K,1990
11th,<=50K,1530
Assoc-voc,<=50K,1455
Masters,>50K,1393
10th,<=50K,1141


#KPI 7

In [35]:
#KPI 7
daf07 = census_clean.groupBy('age group','income').agg({'income':'count'})
display(daf07.sort("count(income)", ascending = False))

age group,income,count(income)
middle,<=50K,12522
young,<=50K,12324
senior,<=50K,6754
middle,>50K,5371
senior,>50K,4284
retired,<=50K,2414
retired,>50K,832
young,>50K,721


#KPI 8

In [37]:
#KPI 8
daf08 = census_clean.groupBy('age group','hr/wk').agg({'income':'count'})
display(daf08.sort("count(income)", ascending = False))

age group,hr/wk,count(income)
middle,Full time,12192
young,Full time,8519
senior,Full time,7506
middle,Overtime,4937
senior,Overtime,2998
young,Temp,2700
retired,Full time,1875
young,Overtime,1826
retired,Temp,859
middle,Temp,764


#KPI 9

In [39]:
#KPI 9
daf09 = census_clean.groupBy(['employer type','income']).agg({'income':'count'})
display(daf09.sort("count(income)", ascending = False))

employer type,income,count(income)
Private,<=50K,26056
Private,>50K,7251
Govt,<=50K,3611
self_emp,<=50K,3471
self_emp,>50K,1971
Govt,>50K,1435
Federal-gov,<=50K,857
Federal-gov,>50K,549
Unemployed,<=50K,19
Unemployed,>50K,2


#KPI 10

In [41]:
#KPI 10
display(census_clean.groupBy(['gender','employer type']).agg({'gender':'count'}))

gender,employer type,count(gender)
Male,self_emp,4626
Male,Govt,3047
Male,Unemployed,14
Male,Federal-gov,961
Female,Federal-gov,445
Female,Govt,1999
Male,Private,21879
Female,Unemployed,7
Female,self_emp,816
Female,Private,11428


#KPI 11

In [43]:
#KPI 11
daf11 = census_clean.groupBy(['employer type','hr/wk']).agg({'hr/wk':'count'})
display(daf11.sort('count(hr/wk)',ascending = False))

employer type,hr/wk,count(hr/wk)
Private,Full time,22963
Private,Overtime,6669
Private,Temp,3675
Govt,Full time,3607
self_emp,Overtime,2449
self_emp,Full time,2403
Federal-gov,Full time,1112
Govt,Overtime,921
self_emp,Temp,590
Govt,Temp,518


#KPI 12

In [45]:
#KPI 12
display(census_clean.groupBy(['gender','income']).agg({'income':'count'}))

gender,income,count(income)
Male,<=50K,20988
Male,>50K,9539
Female,<=50K,13026
Female,>50K,1669


#KPI 13

In [47]:
#KPI 13
daf13 = census_clean.groupBy(['marital','income']).agg({'income':'count'})
display(daf13.sort("count(income)", ascending  = False))

marital,income,count(income)
Never-married,<=50K,13897
Married,<=50K,12007
Married,>50K,9632
Not-Married,<=50K,8110
Not-Married,>50K,875
Never-married,>50K,701


#KPI 14

In [49]:
#KPI 14
display(census_clean.groupBy(['marital','gender']).agg({'income':'count'}))

marital,gender,count(income)
Never-married,Male,8085
Married,Female,2508
Never-married,Female,6513
Not-Married,Female,5674
Not-Married,Male,3311
Married,Male,19131


#KPI 15

In [51]:
#KPI 15
daf15 = census_clean.groupBy(['marital','employer type']).agg({'income':'count'})
display(daf15.sort("count(income)", ascending = False))

marital,employer type,count(income)
Married,Private,14654
Never-married,Private,12011
Not-Married,Private,6642
Married,self_emp,3799
Married,Govt,2447
Never-married,Govt,1417
Not-Married,Govt,1182
Not-Married,self_emp,839
Never-married,self_emp,804
Married,Federal-gov,725


#KPI 16

In [53]:
#KPI 16
#display(census_clean.groupBy(['education','income']).agg({'income':'count'}))
daf16 = census_clean.groupBy(['education','income']).agg({'income':'count'})
display(  daf16.sort("count(income)", ascending=False))

education,income,count(income)
HS-grad,<=50K,12367
Some-college,<=50K,7909
Bachelors,<=50K,4392
Bachelors,>50K,3178
HS-grad,>50K,2416
Some-college,>50K,1990
11th,<=50K,1530
Assoc-voc,<=50K,1455
Masters,>50K,1393
10th,<=50K,1141


#KPI 17

In [55]:
#KPI 17
daf17 = census_clean.groupBy(['occupation','income']).agg({'income':'count'})
display(daf17.sort("count(income)", ascending = False))

occupation,income,count(income)
Adm-clerical,<=50K,4784
Craft-repair,<=50K,4665
Other-service,<=50K,4612
Sales,<=50K,3953
Prof-specialty,<=50K,3304
Exec-managerial,<=50K,3117
Exec-managerial,>50K,2867
Prof-specialty,>50K,2704
Machine-op-inspct,<=50K,2605
Handlers-cleaners,<=50K,1911


In [56]:
#KPI 17
daf017 = census_clean.groupBy(['education','occupation']).agg({'income':'count'})
display(daf017.sort("count(income)", ascending = False))

education,occupation,count(income)
HS-grad,Craft-repair,2882
Bachelors,Prof-specialty,2178
HS-grad,Adm-clerical,2028
Bachelors,Exec-managerial,1977
HS-grad,Other-service,1892
Some-college,Adm-clerical,1833
HS-grad,Sales,1553
HS-grad,Machine-op-inspct,1515
Some-college,Sales,1483
Some-college,Exec-managerial,1277


#KPI 18

In [58]:
daf18 = census_clean.groupBy(['age','income']).agg({'income':'count'})
display(daf18.sort('age'))
#daf.describe()

age,income,count(income)
17,<=50K,493
18,<=50K,695
19,>50K,2
19,<=50K,862
20,<=50K,915
20,>50K,1
21,>50K,6
21,<=50K,929
22,<=50K,1019
22,>50K,15


#KPI 19

In [60]:
daf19 = census_clean.groupBy(['age']).agg({'capitalgain':'mean', 'capitalloss':'mean'})
display(daf19.sort('age'))

age,avg(capitalloss),avg(capitalgain)
17,36.70182555780933,26.75659229208925
18,26.20287769784173,85.41438848920863
19,40.1712962962963,130.32291666666666
20,36.26091703056768,41.66266375545852
21,42.35294117647059,188.1668449197861
22,23.911992263056092,285.7147001934236
23,44.2232070910556,189.42788074133762
24,39.419469026548676,147.40265486725664
25,53.17828773168579,251.65754633715798
26,44.58791208791209,314.6703296703297


#KPI 20

In [62]:
daf20 = census_clean.groupBy(['marital', 'hr/wk']).agg({'income':'count'}).limit(1000)
display(daf20.sort('count(income)',ascending = False ))

marital,hr/wk,count(income)
Married,Full time,13935
Never-married,Full time,9662
Married,Overtime,6522
Not-Married,Full time,6495
Never-married,Temp,2818
Never-married,Overtime,2118
Not-Married,Overtime,1633
Married,Temp,1182
Not-Married,Temp,857


In [63]:
"The actual dataset contained " + str(census.count()) + " rows but after cleaning we are left with " + str(census_clean.count()) + " rows."

Out[59]: 'The actual dataset contained 48842 rows but after cleaning we are left with 45222 rows.'

In [64]:
census_clean.collect()

Out[60]: [Row(age=25, education='11th', occupation='Machine-op-inspct', relationship='Own-child', race='Black', gender='Male', capitalgain=0, capitalloss=0, income='<=50K', age group='young', hr/wk='Full time', region='North America', employer type='Private', marital='Never-married'),
 Row(age=38, education='HS-grad', occupation='Farming-fishing', relationship='Husband', race='White', gender='Male', capitalgain=0, capitalloss=0, income='<=50K', age group='middle', hr/wk='Overtime', region='North America', employer type='Private', marital='Married'),
 Row(age=28, education='Assoc-acdm', occupation='Protective-serv', relationship='Husband', race='White', gender='Male', capitalgain=0, capitalloss=0, income='>50K', age group='young', hr/wk='Full time', region='North America', employer type='Govt', marital='Married'),
 Row(age=44, education='Some-college', occupation='Machine-op-inspct', relationship='Husband', race='Black', gender='Male', capitalgain=7688, capitalloss=0, income='>50K', age group='middle', hr/wk='Full time', region='North America', employer type='Private', marital='Married'),
 Row(age=34, education='10th', occupation='Other-service', relationship='Not-in-family', race='White', gender='Male', capitalgain=0, capitalloss=0, income='<=50K', age group='middle', hr/wk='Full time', region='North America', employer type='Private', marital='Never-married'),
 Row(age=63, education='Prof-school', occupation='Prof-specialty', relationship='Husband', race='White', gender='Male', capitalgain=3103, capitalloss=0, income='>50K', age group='retired', hr/wk='Full time', region='North America', employer type='self_emp', marital='Married'),
 Row(age=24, education='Some-college', occupation='Other-service', relationship='Unmarried', race='White', gender='Female', capitalgain=0, capitalloss=0, income='<=50K', age group='young', hr/wk='Full time', region='North America', employer type='Private', marital='Never-married'),
 Row(age=55, education='7th-8th', occupation='Craft-repair', relationship='Husband', race='White', gender='Male', capitalgain=0, capitalloss=0, income='<=50K', age group='senior', hr/wk='Temp', region='North America', employer type='Private', marital='Married'),
 Row(age=65, education='HS-grad', occupation='Machine-op-inspct', relationship='Husband', race='White', gender='Male', capitalgain=6418, capitalloss=0, income='>50K', age group='retired', hr/wk='Full time', region='North America', employer type='Private', marital='Married'),
 Row(age=36, education='Bachelors', occupation='Adm-clerical', relationship='Husband', race='White', gender='Male', capitalgain=0, capitalloss=0, income='<=50K', age group='middle', hr/wk='Full time', region='North America', employer type='Federal-gov', marital='Married'),
 Row(age=26, education='HS-grad', occupation='Adm-clerical', relationship='Not-in-family', race='White', gender='Female', capitalgain=0, capitalloss=0, income='<=50K', age group='young', hr/wk='Full time', region='North America', employer type='Private', marital='Never-married'),
 Row(age=48, education='HS-grad', occupation='Machine-op-inspct', relationship='Husband', race='White', gender='Male', capitalgain=3103, capitalloss=0, income='>50K', age group='senior', hr/wk='Overtime', region='North America', employer type='Private', marital='Married'),
 Row(age=43, education='Masters', occupation='Exec-managerial', relationship='Husband', race='White', gender='Male', capitalgain=0, capitalloss=0, income='>50K', age group='middle', hr/wk='Overtime', region='North America', employer type='Private', marital='Married'),
 Row(age=20, education='Some-college', occupation='Other-service', relationship='Own-child', race='White', gender='Male', capitalgain=0, capitalloss=0, income='<=50K', age group='young', hr/wk='Temp', region='North America', employer type='Govt', marital='Never-married'),
 Row(age=43, education='HS-grad', occupation='Adm-clerical', relationship='Wife', race='White', gender='Female', capitalgain=0, capitalloss=0, inc

In [65]:
#to view all the string labels in my dataframe
#dtype = [ stype for col,stype in census_clean.dtypes]
for col,dtype in census_clean.dtypes:
  if dtype=='string':
    print(col)
    census_clean.select(col).distinct().show()
    print("----------------------------------------")

education
+------------+
 education|
+------------+
 10th|
 Masters|
 5th-6th|
 Assoc-acdm|
 Assoc-voc|
 7th-8th|
 9th|
 HS-grad|
 Bachelors|
 11th|
 1st-4th|
 Preschool|
 12th|
 Doctorate|
Some-college|
 Prof-school|
+------------+

----------------------------------------
occupation
+-----------------+
 occupation|
+-----------------+
 Sales|
 Exec-managerial|
 Prof-specialty|
Handlers-cleaners|
 Farming-fishing|
 Craft-repair|
 Transport-moving|
 Priv-house-serv|
 Protective-serv|
 Other-service|
 Tech-support|
Machine-op-inspct|
 Armed-Forces|
 Adm-clerical|
+-----------------+

----------------------------------------
relationship
+--------------+
 relationship|
+--------------+
 Own-child|
 Not-in-family|
 Unmarried|
 Wife|
Other-relative|
 Husband|
+--------------+

----------------------------------------
race
+------------------+
 race|
+------------------+
 Other|
Amer-Indian-Eskimo|
 White|
Asian-Pac-Islander|
 Black|
+------------------+

----------------------------------------
gender
+------+
gender|
+------+
Female|
 Male|
+------+

----------------------------------------
income
+------+
income|
+------+
 <=50K|
 >50K|
+------+

----------------------------------------
age group
+---------+
age group|
+---------+
 retired|
 senior|
 young|
 middle|
+---------+

----------------------------------------
hr/wk
+---------+
 hr/wk|
+---------+
 Temp|
Full time|
 Overtime|
+---------+

----------------------------------------
region
+--------------------+
 region|
+--------------------+
 Europe|
Latin & South Ame...|
 North America|
 Others|
 Asia|
+--------------------+

----------------------------------------
employer type
+-------------+
employer type|
+-------------+
 Govt|
 self_emp|
 Private|
 Federal-gov|
 Unemployed|
+-------------+

----------------------------------------
marital
+-------------+
 marital|
+-------------+
 Married|
Never-married|
 Not-Married|
+-------------+

----------------------------------------

In [66]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
categorical_column = [col for col,dtype  in census_clean.dtypes if dtype=='string']
#column = [col for col,dtype  in census_clean.dtypes if dtype=='string']
#String_index = [StringIndexer(inputCol=col, outputCol='index_'+col) for col,dtype  in census_clean.dtypes if dtype=='string'and col!='income']
String_index = [StringIndexer(inputCol=col, outputCol='index_'+col) for col  in categorical_column if col!='income']
String_index += [StringIndexer(inputCol='income', outputCol='label')]
print(String_index)
sample_viz = Pipeline(stages=String_index).fit(census_clean).transform(census_clean)

[StringIndexer_65d2dcadb678, StringIndexer_29851f8a9ffa, StringIndexer_57403f823872, StringIndexer_3dc62df146ff, StringIndexer_d805780be55d, StringIndexer_cdaedcd66fd0, StringIndexer_7bcafd35766f, StringIndexer_71f75003179a, StringIndexer_11b30d948449, StringIndexer_7dd9f96b1154, StringIndexer_0a073f18a6cc]

In [67]:
column = [col for col,dtype  in census_clean.dtypes if dtype=='string' or col in ['age', 'income','capitalgain','capitalloss']]
sample_viz = sample_viz.drop(*column)
sample_viz.printSchema()

root
-- index_education: double (nullable = false)
-- index_occupation: double (nullable = false)
-- index_relationship: double (nullable = false)
-- index_race: double (nullable = false)
-- index_gender: double (nullable = false)
-- index_age group: double (nullable = false)
-- index_hr/wk: double (nullable = false)
-- index_region: double (nullable = false)
-- index_employer type: double (nullable = false)
-- index_marital: double (nullable = false)
-- label: double (nullable = false)

In [68]:
from pyspark.mllib.stat import Statistics
import pandas as pd

# df = sqlCtx.read.format('com.databricks.spark.csv').option('header', 'true').option('inferschema', 'true').load('corr_test.csv')
df = sample_viz
col_names = df.columns
features = df.rdd.map(lambda row: row[0:])
corr_mat=Statistics.corr(features, method="spearman",)
corr_df = pd.DataFrame(corr_mat.round(4))
corr_df.index, corr_df.columns = col_names, col_names
corr_df = corr_df.reset_index()
display(corr_df)


index,index_education,index_occupation,index_relationship,index_race,index_gender,index_age group,index_hr/wk,index_region,index_employer type,index_marital,label
index_education,1.0,-0.0854,-0.038,-0.0019,-0.0195,0.0297,0.0715,0.1231,0.0633,-0.0456,0.1246
index_occupation,-0.0854,1.0,0.109,0.0659,0.0412,0.029,0.0308,0.0328,-0.0796,0.0871,-0.1834
index_relationship,-0.038,0.109,1.0,0.1429,0.606,-0.0436,-0.012,0.0293,-0.0975,0.6601,-0.3407
index_race,-0.0019,0.0659,0.1429,1.0,0.0993,-0.026,-0.0679,0.2231,0.0124,0.0848,-0.081
index_gender,-0.0195,0.0412,0.606,0.0993,1.0,0.0162,-0.016,-0.0116,-0.0465,0.4492,-0.2158
index_age group,0.0297,0.029,-0.0436,-0.026,0.0162,1.0,0.0511,-0.0138,0.0725,0.0081,0.0194
index_hr/wk,0.0715,0.0308,-0.012,-0.0679,-0.016,0.0511,1.0,-0.0377,0.0478,-0.0247,0.0516
index_region,0.1231,0.0328,0.0293,0.2231,-0.0116,-0.0138,-0.0377,1.0,-0.0491,-0.0405,-0.0368
index_employer type,0.0633,-0.0796,-0.0975,0.0124,-0.0465,0.0725,0.0478,-0.0491,1.0,-0.0825,0.1121
index_marital,-0.0456,0.0871,0.6601,0.0848,0.4492,0.0081,-0.0247,-0.0405,-0.0825,1.0,-0.3972
